<a href="https://colab.research.google.com/github/Astatine404/IFT6390_KaggleComp1/blob/main/OtherMethods_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sklearn
import numpy as np
import urllib.request
import pandas as pd

In [ ]:
train_data = np.loadtxt(urllib.request.urlopen("https://transfer.sh/dlcC5z/train.csv"), delimiter=",", skiprows=1)
train_data = train_data [:, 1:]
#train_data = pd.DataFrame(train_data).drop_duplicates().to_numpy()

In [ ]:
train_data.shape

(47760, 20)

In [ ]:
train_features = train_data[:, :-1]
train_labels = train_data[:, -1]
pd.DataFrame(data=train_features)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,102326.2344,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804.0
1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,100803.1094,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006.0
2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,101885.3906,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331.0
3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,100824.6328,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906.0
4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,102553.8984,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47755,-30.625815,353.4375,14.680613,3.153518,4.250256,-0.406992,3.480553,0.008333,102109.4844,102109.4844,213.804184,259.846222,2.470000e-09,291.791870,290.141663,177.677322,12058.16016,63.678669,20001106.0
47756,-24.993481,242.1875,29.540585,9.181329,-0.369805,7.125068,-5.466469,0.013645,101734.2813,101734.2813,213.867416,263.996307,2.620000e-09,296.322327,296.220642,150.108871,12272.25781,65.238327,20010712.0
47757,-30.625815,353.4375,20.991110,7.709649,-6.284292,3.702500,-7.359541,0.010529,101800.8516,101800.8516,216.394852,264.192566,4.240000e-09,290.757996,290.736847,152.666443,12232.26953,63.929737,20011029.0
47758,23.585398,276.5625,45.689816,-3.018448,0.280351,-6.938124,0.681055,0.020653,101641.2266,101641.2266,217.026199,265.778900,8.620000e-08,302.751862,301.542786,145.255554,12404.29297,66.656548,20020712.0


In [ ]:
from sklearn.model_selection import train_test_split

# split dataset to train and test
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, 
                                                    test_size=0.2, 
                                                    shuffle=True, 
                                                    stratify=train_labels,
                                                    random_state=45
                                                    )  
X_train_scaled = X_train
X_test_scaled = X_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, IsolationForest, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10],
    'max_features': [5, 6, 7, 8],
    'min_samples_leaf': [1],
    'min_samples_split': [4],
    'n_estimators': [100],
    'criterion' :['gini']
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train_scaled, y_train)
grid_search.best_params_

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.0min finished


{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 6,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 100}

In [ ]:
classifier = RandomForestClassifier(n_estimators=100, min_samples_split=150, max_depth=55)
#classifier3 = RandomForestClassifier(n_estimators=500, max_depth=10, max_features=6, criterion='gini', min_samples_split=4)

classifier.fit(X_train_scaled, y_train) #final 0.7598

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=55, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=150,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
print(classifier.score(X_train_scaled, y_train))
print(classifier.score(X_test_scaled, y_test))

0.8523246786854136
0.8424524641055491


In [ ]:
kaggle_data = np.loadtxt(urllib.request.urlopen("http://transfer.sh/QUScgs/test.csv"), delimiter=",", skiprows=1)
kaggle_data = kaggle_data[:, 1:]
#kaggle_inputs_scaled = minmax_scale(kaggle_data)

In [ ]:
#pred = classifier.predict(kaggle_inputs_scaled[:, feat_selector.get_support(indices=True)])
pred = classifier.predict(kaggle_data)
print(pred[pred==0].size / pred.size, pred[pred==1].size / pred.size, pred[pred==2].size / pred.size)

0.639207650273224 0.1284153005464481 0.23237704918032787


In [ ]:
indices = np.array(list(i for i in range(pred.size)))
result = np.empty((pred.size, 2))
result[:, 0] = indices
result[:, 1] = pred

In [ ]:
pd.DataFrame(data=result).tail(10)

,0,1
7310,7310.0,1.0
7311,7311.0,1.0
7312,7312.0,1.0
7313,7313.0,1.0
7314,7314.0,1.0
7315,7315.0,1.0
7316,7316.0,1.0
7317,7317.0,1.0
7318,7318.0,1.0
7319,7319.0,1.0


In [ ]:
np.savetxt('result5.csv', result, fmt="%d", delimiter=",")

In [ ]:
!curl --upload-file ./result5.csv http://transfer.sh/result5.csv

http://transfer.sh/4VcIFf/result5.csv

# K-Fold Cross

In [ ]:
# -------- Cross validate model with Kfold stratified cross val ---------------
from sklearn.model_selection import StratifiedKFold, cross_val_score

kfold = StratifiedKFold(n_splits=10)

# Modeling step Test differents algorithms
# classifiers = [LogisticRegression(),
#                RandomForestClassifier()]
classifiers = [AdaBoostClassifier()]
results = []
for model in classifiers :
    results.append(cross_val_score(model, X_train_scaled, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std})

In [ ]:
cv_res

,CrossValMeans,CrossValerrors
0,0.785909,0.000022
1,0.861940,0.004643


In [ ]:
cv_res

,CrossValMeans,CrossValerrors
0,0.739636,0.009372


In [ ]:
classifiers = [('rf', RandomForestClassifier(n_estimators=100, min_samples_split=150, max_depth=55)), 
               ('xgb', XGBClassifier(n_estimators=110, max_depth=5, colsample_bytree=0.8, eta=0.05, min_child_weight=3, 
                   gamma=0, subsample=0.8, scale_pos_weight=1)), 
               ('gbc', GradientBoostingClassifier(n_estimators=200))]
ensemble = VotingClassifier(estimators=classifiers)
ensemble.fit(X_train_scaled, y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=55,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=150,
                                                     min_weight_fraction_leaf=0.0,
        

In [ ]:
print(classification_report(y_test, ensemble.predict(X_test_scaled), digits=4))
confusion_matrix(y_test, ensemble.predict(X_test_scaled))

              precision    recall  f1-score   support

         0.0     0.8650    0.9497    0.9054      1930
         1.0     0.6795    0.4344    0.5300       122
         2.0     0.7868    0.5695    0.6608       525

    accuracy                         0.8479      2577
   macro avg     0.7771    0.6512    0.6987      2577
weighted avg     0.8403    0.8479    0.8378      2577



array([[1833,   19,   78],
       [  66,   53,    3],
       [ 220,    6,  299]])

In [ ]:
pred = ensemble.predict(kaggle_data)
print(pred[pred==0].size / pred.size, pred[pred==1].size / pred.size, pred[pred==2].size / pred.size)

0.8437158469945355 0.041666666666666664 0.11461748633879781


In [ ]:
indices = np.array(list(i for i in range(pred.size)))
result = np.empty((pred.size, 2))
result[:, 0] = indices
result[:, 1] = pred

In [ ]:
pd.DataFrame(data=result).tail(10)

,0,1
7310,7310.0,0.0
7311,7311.0,0.0
7312,7312.0,1.0
7313,7313.0,1.0
7314,7314.0,1.0
7315,7315.0,0.0
7316,7316.0,1.0
7317,7317.0,1.0
7318,7318.0,1.0
7319,7319.0,1.0


In [ ]:
np.savetxt('result4.csv', result, fmt="%d", delimiter=",")

In [ ]:
!curl --upload-file ./result4.csv http://transfer.sh/result4.csv

http://transfer.sh/htJGPB/result4.csv

In [ ]:
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(solver= 'lbfgs', random_state= 0, max_iter=1000, hidden_layer_sizes= 10, alpha= 1e-07)
nn.fit(X_train_scaled, y_train)
print(classification_report(y_test, nn.predict(X_test_scaled), digits=4))
confusion_matrix(y_test, nn.predict(X_test_scaled))

              precision    recall  f1-score   support

         0.0     0.8035    0.8893    0.8443      4392
         1.0     0.5905    0.5300    0.5586       400
         2.0     0.6968    0.5155    0.5926      1645

    accuracy                         0.7715      6437
   macro avg     0.6970    0.6449    0.6652      6437
weighted avg     0.7630    0.7715    0.7622      6437



array([[3906,  118,  368],
       [ 187,  212,    1],
       [ 768,   29,  848]])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

parameters = {'solver': ['lbfgs'], 'max_iter': [1000,1400,1900,2000], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15)}
clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)

clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.best_params_)

0.04192839195979899
{'alpha': 1e-07, 'hidden_layer_sizes': 10, 'max_iter': 1000, 'solver': 'lbfgs'}


In [ ]:
from sklearn.model_selection import GridSearchCV

#create a new random forest classifier
rf = GradientBoostingClassifier()
#create a dictionary of all values we want to test for n_estimators
params_rf = {'n_estimators': [10, 50, 100, 200, 400]}
#use gridsearch to test all values for n_estimators
rf_gs = GridSearchCV(rf, params_rf, cv=5, n_jobs=-1)
#fit model to training data
rf_gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

In [ ]:
print(rf_gs.best_params_)

{'n_estimators': 400}


In [ ]:
knn = GradientBoostingClassifier(n_estimators=200)
knn.fit(X_train_scaled, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print(classification_report(y_test, knn.predict(X_test_scaled), digits=4))
confusion_matrix(y_test, knn.predict(X_test_scaled))

              precision    recall  f1-score   support

         0.0     0.8484    0.8611    0.8547      1930
         1.0     0.5192    0.4426    0.4779       122
         2.0     0.5486    0.5371    0.5428       525

    accuracy                         0.7753      2577
   macro avg     0.6388    0.6136    0.6251      2577
weighted avg     0.7717    0.7753    0.7733      2577



array([[1662,   43,  225],
       [  61,   54,    7],
       [ 236,    7,  282]])

In [ ]:
from sklearn.svm import SVC

nn = SVC(C=0.5)
nn.fit(X_train_scaled, y_train)
print(classification_report(y_test, nn.predict(X_test_scaled), digits=4))
confusion_matrix(y_test, nn.predict(X_test_scaled))

              precision    recall  f1-score   support

         0.0     0.8144    0.8939    0.8523      4392
         1.0     0.8012    0.3225    0.4599       400
         2.0     0.6900    0.6103    0.6477      1645

    accuracy                         0.7859      6437
   macro avg     0.7685    0.6089    0.6533      6437
weighted avg     0.7818    0.7859    0.7756      6437



array([[3926,   20,  446],
       [ 266,  129,    5],
       [ 629,   12, 1004]])

In [ ]:
from xgboost.sklearn import XGBClassifier

nn = XGBClassifier(n_estimators=110, max_depth=5, colsample_bytree=0.8, eta=0.05, min_child_weight=3, 
                   gamma=0, subsample=0.8, scale_pos_weight=1)
nn.fit(X_train_scaled, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.05, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=3, missing=None, n_estimators=110, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [ ]:
from xgboost.sklearn import XGBClassifier

nn = XGBClassifier(n_estimators=100, max_depth=10, alpha=20, scale_pos_weight=1, learning_rate=0.01)
nn.fit(X_train_scaled, y_train)

XGBClassifier(alpha=20, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, nn.predict(X_test_scaled), digits=4))
confusion_matrix(y_test, nn.predict(X_test_scaled))

              precision    recall  f1-score   support

         0.0     0.9139    0.9462    0.9298      7507
         1.0     0.7685    0.6475    0.7028       400
         2.0     0.7512    0.6590    0.7021      1645

    accuracy                         0.8842      9552
   macro avg     0.8112    0.7509    0.7782      9552
weighted avg     0.8798    0.8842    0.8811      9552



array([[7103,   56,  348],
       [ 130,  259,   11],
       [ 539,   22, 1084]])

In [ ]:
print(nn.score(X_train_scaled, y_train))
print(nn.score(X_test_scaled, y_test))

0.9104899497487438
0.884212730318258


In [ ]:
from sklearn.tree import DecisionTreeClassifier

nn = DecisionTreeClassifier()
nn.fit(X_train_scaled, y_train)
print(classification_report(y_test, nn.predict(X_test_scaled), digits=4))
confusion_matrix(y_test, nn.predict(X_test_scaled))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}
xgbr = XGBClassifier(seed = 20)
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=10,
                         verbose=1,
                         n_jobs=-1)
clf.fit(X_train_scaled, y_train)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 25.9min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 29.0min finished


Best parameters: {'subsample': 0.5, 'n_estimators': 1000, 'max_depth': 5, 'colsample_bytree': 0.8999999999999999, 'colsample_bylevel': 0.5}
Lowest RMSE:  0.7337306284840065


In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
classifier = LGBMClassifier(n_estimators=5000, early_stopping_rounds=30)
classifier.fit(X_train_scaled, y_train, eval_set=(X_test, y_test))
classifier.score(X_test_scaled, y_test)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 0.637021
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's multi_logloss: 0.602899
[3]	valid_0's multi_logloss: 0.574264
[4]	valid_0's multi_logloss: 0.551522
[5]	valid_0's multi_logloss: 0.531609
[6]	valid_0's multi_logloss: 0.514072
[7]	valid_0's multi_logloss: 0.499389
[8]	valid_0's multi_logloss: 0.486382
[9]	valid_0's multi_logloss: 0.473795
[10]	valid_0's multi_logloss: 0.462516
[11]	valid_0's multi_logloss: 0.452078
[12]	valid_0's multi_logloss: 0.443036
[13]	valid_0's multi_logloss: 0.434642
[14]	valid_0's multi_logloss: 0.426421
[15]	valid_0's multi_logloss: 0.419335
[16]	valid_0's multi_logloss: 0.41188
[17]	valid_0's multi_logloss: 0.405751
[18]	valid_0's multi_logloss: 0.399889
[19]	valid_0's multi_logloss: 0.394038
[20]	valid_0's multi_logloss: 0.388941
[21]	valid_0's multi_logloss: 0.383659
[22]	valid_0's multi_logloss: 0.379112
[23]	valid_0's multi_logloss: 0.374494
[24]	valid_0's multi_logloss: 0.370347
[25]	valid_0

0.8206869784591501

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, classifier.predict(X_test_scaled), digits=4))
confusion_matrix(y_test, classifier.predict(X_test_scaled))

              precision    recall  f1-score   support

         0.0     0.8722    0.9088    0.8901      3859
         1.0     0.6452    0.4918    0.5581       244
         2.0     0.6702    0.6038    0.6353      1050

    accuracy                         0.8269      5153
   macro avg     0.7292    0.6681    0.6945      5153
weighted avg     0.8203    0.8269    0.8225      5153



array([[3507,   52,  300],
       [ 112,  120,   12],
       [ 402,   14,  634]])

In [ ]:
!pip install lightgbm --install-option=--gpu

/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 1.5 MB 13.1 MB/s 
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from lightgbm import LGBMClassifier
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

n_HP_points_to_test = 100

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

clf = LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000, device='gpu')
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='f1_weighted',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'multi_logloss', 
            "eval_set" : [(X_test_scaled,y_test)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 100,
            'categorical_feature': 'auto'}
gs.fit(X_train_scaled, y_train, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Training until validation scores don't improve for 30 rounds.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[100]	valid's multi_logloss: 0.349976
[200]	valid's multi_logloss: 0.307828
[300]	valid's multi_logloss: 0.29209
[400]	valid's multi_logloss: 0.285548
[500]	valid's multi_logloss: 0.283555
Early stopping, best iteration is:
[543]	valid's multi_logloss: 0.283224
Training until validation scores don't improve for 30 rounds.
[100]	valid's multi_logloss: 0.350224
[200]	valid's multi_logloss: 0.306207
[300]	valid's multi_logloss: 0.288379
[400]	valid's multi_logloss: 0.280142
[500]	valid's multi_logloss: 0.276995
Early stopping, best iteration is:
[537]	valid's multi_logloss: 0.276812
Training until validation scores don't improve for 30 rounds.
[100]	valid's multi_logloss: 0.352777
[200]	valid's multi_logloss: 0.311016
[300]	valid's multi_logloss: 0.293509
[400]	valid's multi_logloss: 0.286402
[500]	valid's multi_logloss: 0.284517
Early stopping, best iteration is:
[513]	valid's multi_logloss: 0.284203
Training until validation scores don't improve for 30 rounds.
[100]	valid's multi_loglos

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 46.1min finished


Training until validation scores don't improve for 30 rounds.
[100]	valid's multi_logloss: 0.314002
[200]	valid's multi_logloss: 0.283902
[300]	valid's multi_logloss: 0.275033
[400]	valid's multi_logloss: 0.273441
Early stopping, best iteration is:
[380]	valid's multi_logloss: 0.273427
Best score reached: 0.8332137987212981 with params: {'colsample_bytree': 0.9492410177317588, 'min_child_samples': 243, 'min_child_weight': 1, 'num_leaves': 25, 'reg_alpha': 10, 'reg_lambda': 0.1, 'subsample': 0.4592512443442188} 


In [ ]:
classifier = LGBMClassifier(n_estimators=5000, early_stopping_rounds=200, colsample_bytree=0.9492410177317588, min_child_samples=243, min_child_weight=1, num_leaves=25,reg_alpha=10, reg_lambda= 0.1,
                            subsample=0.4592512443442188, max_depth=-1, random_state=314, learning_rate=0.02, class_weight='balanced')
classifier.fit(X_train_scaled, y_train, eval_set=(X_test, y_test))
classifier.score(X_test_scaled, y_test)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 1.08722
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's multi_logloss: 1.07629
[3]	valid_0's multi_logloss: 1.06596
[4]	valid_0's multi_logloss: 1.056
[5]	valid_0's multi_logloss: 1.04616
[6]	valid_0's multi_logloss: 1.03667
[7]	valid_0's multi_logloss: 1.02753
[8]	valid_0's multi_logloss: 1.01868
[9]	valid_0's multi_logloss: 1.01007
[10]	valid_0's multi_logloss: 1.00143
[11]	valid_0's multi_logloss: 0.993048
[12]	valid_0's multi_logloss: 0.984991
[13]	valid_0's multi_logloss: 0.977333
[14]	valid_0's multi_logloss: 0.969755
[15]	valid_0's multi_logloss: 0.962241
[16]	valid_0's multi_logloss: 0.955083
[17]	valid_0's multi_logloss: 0.947817
[18]	valid_0's multi_logloss: 0.940867
[19]	valid_0's multi_logloss: 0.933975
[20]	valid_0's multi_logloss: 0.927434
[21]	valid_0's multi_logloss: 0.920861
[22]	valid_0's multi_logloss: 0.91438
[23]	valid_0's multi_logloss: 0.908298
[24]	valid_0's multi_logloss: 0.902411
[25]	valid_0's multi_lo

0.859305259072385

In [ ]:
from lightgbm import LGBMClassifier

classifier1 = LGBMClassifier(n_estimators=5000, early_stopping_rounds=200, colsample_bytree=0.9492410177317588, min_child_samples=243, min_child_weight=1, num_leaves=25,reg_alpha=10, reg_lambda= 0.1,
                            subsample=0.4592512443442188, max_depth=-1, random_state=314, learning_rate=0.02)
classifier1.fit(X_train_scaled, y_train, eval_set=(X_test, y_test))
classifier1.score(X_test_scaled, y_test)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 0.676773
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's multi_logloss: 0.669007
[3]	valid_0's multi_logloss: 0.661903
[4]	valid_0's multi_logloss: 0.655127
[5]	valid_0's multi_logloss: 0.648597
[6]	valid_0's multi_logloss: 0.642281
[7]	valid_0's multi_logloss: 0.63635
[8]	valid_0's multi_logloss: 0.630658
[9]	valid_0's multi_logloss: 0.625288
[10]	valid_0's multi_logloss: 0.619959
[11]	valid_0's multi_logloss: 0.614828
[12]	valid_0's multi_logloss: 0.609917
[13]	valid_0's multi_logloss: 0.605136
[14]	valid_0's multi_logloss: 0.600485
[15]	valid_0's multi_logloss: 0.595903
[16]	valid_0's multi_logloss: 0.591573
[17]	valid_0's multi_logloss: 0.587302
[18]	valid_0's multi_logloss: 0.583165
[19]	valid_0's multi_logloss: 0.579249
[20]	valid_0's multi_logloss: 0.575491
[21]	valid_0's multi_logloss: 0.571846
[22]	valid_0's multi_logloss: 0.568322
[23]	valid_0's multi_logloss: 0.564783
[24]	valid_0's multi_logloss: 0.561289
[25]	valid_

0.8199456732634847

In [ ]:
from lightgbm import LGBMClassifier

classifier2 = LGBMClassifier(n_estimators=5000, early_stopping_rounds=200, colsample_bytree=0.9492410177317588, min_child_samples=243, min_child_weight=1, num_leaves=25,reg_alpha=10, reg_lambda= 0.1,
                            subsample=0.4592512443442188, max_depth=-1, random_state=314, learning_rate=0.02, class_weight='balanced')
classifier2.fit(X_train_scaled, y_train, eval_set=(X_test, y_test))
classifier2.score(X_test_scaled, y_test)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 1.0876
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's multi_logloss: 1.0768
[3]	valid_0's multi_logloss: 1.06639
[4]	valid_0's multi_logloss: 1.05638
[5]	valid_0's multi_logloss: 1.04651
[6]	valid_0's multi_logloss: 1.0369
[7]	valid_0's multi_logloss: 1.02737
[8]	valid_0's multi_logloss: 1.01865
[9]	valid_0's multi_logloss: 1.00977
[10]	valid_0's multi_logloss: 1.00118
[11]	valid_0's multi_logloss: 0.992845
[12]	valid_0's multi_logloss: 0.98463
[13]	valid_0's multi_logloss: 0.976921
[14]	valid_0's multi_logloss: 0.969085
[15]	valid_0's multi_logloss: 0.961682
[16]	valid_0's multi_logloss: 0.954431
[17]	valid_0's multi_logloss: 0.947201
[18]	valid_0's multi_logloss: 0.940304
[19]	valid_0's multi_logloss: 0.933635
[20]	valid_0's multi_logloss: 0.927121
[21]	valid_0's multi_logloss: 0.920681
[22]	valid_0's multi_logloss: 0.914352
[23]	valid_0's multi_logloss: 0.908251
[24]	valid_0's multi_logloss: 0.902293
[25]	valid_0's multi_log

0.8692277842452464

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, classifier2.predict(X_test_scaled), digits=4))

              precision    recall  f1-score   support

         0.0     0.9908    0.8378    0.9079      1930
         1.0     0.6250    0.9836    0.7643       122
         2.0     0.6680    0.9581    0.7872       525

    accuracy                         0.8692      2577
   macro avg     0.7613    0.9265    0.8198      2577
weighted avg     0.9077    0.8692    0.8765      2577



In [ ]:
# Soft Ensemble
def custom_predict(clf1, clf2, X_test):
    p1 = clf1.predict_proba(X_test)
    p2 = clf2.predict_proba(X_test)
    p1[:, 0] = p1[:, 0] * 1.7
    p2[:, 1] = p2[:, 1] * 0.9
    p2[:, 2] = p2[:, 2] * 0.9
    pred = np.argmax(p1 + p2, axis=1)
    #pred[np.argmax(p1, axis=1) == 0] = 0
    return pred

In [ ]:
print(classification_report(y_test, custom_predict(classifier, classifier2, X_test_scaled), digits=4))

              precision    recall  f1-score   support

         0.0     0.8845    0.9244    0.9040      1930
         1.0     0.6316    0.5902    0.6102       122
         2.0     0.7466    0.6343    0.6859       525

    accuracy                         0.8494      2577
   macro avg     0.7542    0.7163    0.7333      2577
weighted avg     0.8444    0.8494    0.8456      2577



In [ ]:
print(classification_report(y_test, custom_predict(classifier1, classifier2, X_test_scaled), digits=4))

              precision    recall  f1-score   support

         0.0     0.8964    0.8834    0.8899      1930
         1.0     0.5845    0.6803    0.6288       122
         2.0     0.6717    0.6819    0.6767       525

    accuracy                         0.8328      2577
   macro avg     0.7175    0.7486    0.7318      2577
weighted avg     0.8359    0.8328    0.8341      2577



In [ ]:
kaggle_data = np.loadtxt(urllib.request.urlopen("https://transfer.sh/lCFfeo/test.csv"), delimiter=",", skiprows=1)
kaggle_data = kaggle_data[:, 1:]
#pred = custom_predict(classifier, classifier2, kaggle_data)
pred = nn.predict(kaggle_data)
print(pred[pred==0].size / pred.size, pred[pred==1].size / pred.size, pred[pred==2].size / pred.size)
indices = np.array(list(i for i in range(pred.size)))
result = np.empty((pred.size, 2))
result[:, 0] = indices
result[:, 1] = pred
pd.DataFrame(data=result).tail(10)

0.8006830601092896 0.036065573770491806 0.16325136612021857


,0,1
7310,7310.0,0.0
7311,7311.0,1.0
7312,7312.0,1.0
7313,7313.0,1.0
7314,7314.0,1.0
7315,7315.0,0.0
7316,7316.0,1.0
7317,7317.0,1.0
7318,7318.0,1.0
7319,7319.0,1.0


In [ ]:
np.savetxt('result5.csv', result, fmt="%d", delimiter=",")
!curl --upload-file ./result5.csv http://transfer.sh/result5.csv

http://transfer.sh/1FmQEf/result5.csv